# Understanding Retrieval Question Answering

#### Install dependencies

In [1]:
%pip install -Uqqq rich openai tiktoken wandb
%pip install langchain langchain-core langchain-community langchain-experimental
%pip install unstructured==0.5.6
%pip install python-magic-bin
%pip install unstructured[md]
%pip install langchain-chroma
%pip install langchain-openai
%pip install wget

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Import libraries

In [2]:
import os, random, wget, shutil
from pathlib import Path
import tiktoken
from rich.markdown import Markdown
import wandb

#### Loading OPENAI API key

In [3]:
if os.getenv('OPENAI_API_KEY') is None:
    if any(['VSCODE' in x for x in os.environ.keys()]):
        print('Plase enter password in the VS Code prompt at the top of your VS Code window!')
    os.environ['OPENAI_API_KEY'] == getpass('Paste your OpenAI key from: https://paltform.openai.com/account/api-key\n')

assert os.getenv('OPENAI_API_KEY',''.startswith('sk-')), "This doesn't look like a valid OpenAI API key"
print('OpenAI API key configured')

OpenAI API key configured


#### Download necessary files

In [4]:
class Utils:
    # Function to clone the repository
    def clone_repo(self, repo_url, target_dir):
        if not os.path.exists(target_dir):
            os.system(f'git config --global http.postBuffer 1048576000')
            os.system(f'git clone --depth=1 {repo_url} {target_dir}')
        else:
            print(f"Directory {target_dir} already exists.")

    # Function to get file absolute path
    def get_file_name(self, dir):
        file_list = []
        for file in os.listdir(dir):
            if os.path.isfile(os.path.join(dir, file)):
                file_list.append(file)
        return file_list

    # Function to copy files from one directory to another
    def copy_dir(self, src_dir, dst_dir):
        file_list = self.get_file_name(src_dir)
        if not os.path.isdir(dst_dir):
            os.makedirs(dst_dir)
        if len(file_list) != 0:
            for file_name in file_list:
                src = os.path.join(src_dir, file_name)
                dst = os.path.join(dst_dir, file_name)
                print(f"Copying {src} to {dst}")
                self.move_or_copy_file(src, dst, False)

    # Function to move one file from one directory to another
    def move_or_copy_file(self, src_file, dst_file, move_file=True):
        try:
            dst_dir = dst_file.split('/')[:-1]
            dst_dir = '/'.join(dst_dir)
            if not os.path.isdir(dst_dir):
                os.makedirs(dst_dir)
            if move_file:
                shutil.move(src_file, dst_file)
                print(f"{src_file} moved successfully.")
            else:
                shutil.copy(src_file, dst_file)
                print(f"{src_file} copied successfully.")
        # If source and destination are same
        except shutil.SameFileError:
            print("Source and destination represent the same file.")
        # If there is any permission issue
        except PermissionError:
            print("Permission denied.")
        # For other errors
        except Exception as e:
            print(f"Error occurred while copying file: {e}")


In [8]:
# Download files on colab
for file_name in ['examples.txt','prompt_template.txt','system_template.txt']:
    if not Path("../files/"+file_name).exists():
        downloaded_file = wget.download(f'https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/{file_name}')
        Utils().move_or_copy_file(f'{file_name}', f'../files/{file_name}')

examples.txt moved successfully.
prompt_template.txt moved successfully.


In [6]:
Utils().clone_repo('https://github.com/wandb/edu.git', 'edu')
Utils().copy_dir(src_dir='edu/llm-apps-course/docs_sample', dst_dir='../files/docs_sample')

Directory edu already exists.
Copying edu/llm-apps-course/docs_sample\collaborate-on-reports.md to ./files/docs_sample\collaborate-on-reports.md
edu/llm-apps-course/docs_sample\collaborate-on-reports.md copied successfully.
Copying edu/llm-apps-course/docs_sample\dataset-versioning.md to ./files/docs_sample\dataset-versioning.md
edu/llm-apps-course/docs_sample\dataset-versioning.md copied successfully.
Copying edu/llm-apps-course/docs_sample\getting-started.md to ./files/docs_sample\getting-started.md
edu/llm-apps-course/docs_sample\getting-started.md copied successfully.
Copying edu/llm-apps-course/docs_sample\lightning.md to ./files/docs_sample\lightning.md
edu/llm-apps-course/docs_sample\lightning.md copied successfully.
Copying edu/llm-apps-course/docs_sample\model-management-concepts.md to ./files/docs_sample\model-management-concepts.md
edu/llm-apps-course/docs_sample\model-management-concepts.md copied successfully.
Copying edu/llm-apps-course/docs_sample\quickstart.md to ./file

#### Enable W&B to track our experiment

In [9]:
# we need a single line of code to start tracing langchain with W&B
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
# start logging to W&B
os.environ['WANDB_NOTEBOOK_NAME'] = 'Retrieval.ipynb'
# autolog(init={'project':'llmapps', 'job_type': 'generation'})
wandb.init(project='llmapp', job_type='retrieval')

wandb: Currently logged in as: mary1378. Use `wandb login --relogin` to force relogin


## Langchain
LangChain is a framework for developing applications powered by language models. We will use some of its features in the code below. Let's start by configuring W&B tracing.

## Parsing documents
We will use a small sample of markdown documents in this notebook. Let's find them and make sure we can stuff them into the prompt. That means they may need to be chuncked and not exceed some number of tokens.

In [10]:
MODEL_NAME = 'gpt-3.5-turbo'

In [11]:
from langchain_community.document_loaders import DirectoryLoader

def find_md_files(directory):
    'Find all markdown files in a directory and return a LangChain Document'
    dl = DirectoryLoader(directory, '**/*.md')
    return dl.load()

documents = find_md_files('../files/docs_sample/')
len(documents), documents

(11,
 [Document(metadata={'source': '..\\files\\docs_sample\\collaborate-on-reports.md'}, page_content='--\ndescription: Collaborate and share W&B Reports with peers, co-workers, and your team.\n---\n\n# Collaborate on reports'),
  Document(metadata={'source': '..\\files\\docs_sample\\dataset-versioning.md'}, page_content='--\ndescription: Guide to using Artifacts for dataset versioning\n---\n\n# Dataset Versioning'),
  Document(metadata={'source': '..\\files\\docs_sample\\getting-started.md'}, page_content='--\n\ndescription: Getting started guide for W&B Launch.\n\n--\n\n# Getting started\n\nFollow this guide to get started using W&B Launch. This guide will walk you through the setup of the fundamental components of a launch workflow: a **job**, **launch queue**, and **launch queue**.\n\nA **job** is a reusable blueprint for configuring and executing a step of your ML workflow. Jobs can be automatically captured from your workloads when your track those workloads with W&B. In this gu

In [12]:
# We will need to count tokens in the documents, and for that we need the tokenizer
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)

In [13]:
def count_tokens(documents):
    token_counts = [len(tokenizer.encode(document.page_content)) for document in documents]
    return token_counts

count_tokens(documents)

[26, 18, 2845, 4000, 18, 29, 335, 653, 1818, 1568, 1061]

We will use `LangChain` built in `MarkdownTextSplitter` to split the documents into sections. Actually splitting Markdown without breaking syntax is not that easy. This splitter strips out syntax.
- We can pass the `chunk_size` param and avoid lenghty chunks.
- The `chunk_overlap` param is useful so you don't cut sentences randomly. This is less necessary with Markdown

The `MarkdownTextSplitter` also takes care of removing double line breaks and save us some tokens that way.

In [14]:
from langchain.text_splitter import MarkdownTextSplitter

md_text_splitter = MarkdownTextSplitter(chunk_size=1000)
document_sections = md_text_splitter.split_documents(documents)
len(document_sections), max(count_tokens(document_sections))

(81, 382)

Let's look at the first section of our documents

In [18]:
Markdown(document_sections[10].page_content)

wandb:                                                                                                             

wandb: 🚀 View run comic-firebrand-342 at: https://wandb.ai/username/launch-quickstart/runs/p89pnj2u               

wandb: Synced 5 W&B file(s), 1 media file(s), 3 artifact file(s) and 1 other file(s)                               

wandb: Find logs at: ./wandb/run-20230323_120437-p89pnj2u/logs

## Embeddings
Let's now use [LLM embeddings](https://irisagent.com/blog/understanding-llm-embeddings-a-comprehensive-guide/) with a [vector database retriever](https://www.analyticsvidhya.com/blog/2023/07/guide-to-chroma-db-a-vector-store-for-your-generative-ai-llms/) to find relevant documents for a query.
If you are curious about using CHroma DB from langchain see this [link](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/).

In [19]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# We will use the OpenAIEmbeddings to embed the text, and Chroma to store the vectors
embeddings =  OpenAIEmbeddings()
db = Chroma.from_documents(document_sections, embeddings)

: 

We can create a retriever from the db now, we can pass the k param to get the most relevant sections from the similarity search

In [ ]:
# k is the number of sections to be retrieved that are the most relevant sections
retriever = db.as_retriever(search_kargs=dict(k=3))

In [ ]:
query = 'How can I share my W&B report with my team members in a public W&B project?'
docs = retriever.invoke(query)

In [ ]:
# Let's see the results
for doc in docs:
    print(doc.metadata['source'])

files/docs_sample/collaborate-on-reports.md
files/docs_sample/teams.md
files/docs_sample/teams.md
files/docs_sample/teams.md


In [ ]:
for doc in docs:
    print(doc.page_content)

--
description: Collaborate and share W&B Reports with peers, co-workers, and your team.
---

# Collaborate on reports
Team Trials

W&B offers free trials for business teams, no credit card required. During the trial, you and your colleagues will have access to all the features in W&B. Once the trial is over, you can upgrade your plan to continue using a W&B Team to collaborate. Your personal W&B account will always remain free, and if you're a student or teacher you can enroll in an academic plan.

See the pricing page for more information on our plans. You can download all your data at any time, either using the dashboard UI or via our Export API.

Change the account settings for an organization

If you're a paid user, then you can go to your 'Subscriptions' page and click on the three dots next to the 'Account' next to your organization name. You'll be then able to edit the billing info for your organization, add seats to your org or contact sales to upgrade your plan.
description: 

## Stuff Prompt
We'll take the content of the retrieved documents, stuff them into prompt template along with the query, and pass into an LLM to obtain the answer

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=['context', 'question']
)
context = '\n\n'.join(doc.page_content for doc in docs)
prompt = PROMPT.format(context=context, question=query)

Use langchain to call openai chat API with the question

In [ ]:
from langchain_openai import OpenAI
llm = OpenAI()
response = llm.invoke(prompt)
Markdown(response)

To share your W&B report with your team members in a public W&B project, you can invite them to your team and then 
add them as collaborators on the project. Additionally, you can customize your team's profile page to showcase the 
report and make it visible to the public or team members.

## Using Langchain

Langchain gives us tools to do this efficiently in few lines of code. Lets do the same using `RetrievalQA` chain.

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type='stuff', retriever=retriever)
result = qa.invoke(query)
Markdown(result['result'])

To share your W&B report with your team members in a public W&B project, you can create a team profile by          
customizing your team's profile page. This will allow you to showcase reports and projects that are visible to the 
public or team members. You can also share external links on your team profile. Additionally, you can invite team  
members to join your team and give them access to view and collaborate on your reports.

In [ ]:
import wandb
wandb.finish()